# Query Expansion

0) Just some imports

In [1]:
import re
import math
import numpy as np
import common as cm

# 1) Simple search engine (for 3.0)

1.1) Get acquainted with the below class. There are several TODOs. However, DO NOT complete them now. 

In [2]:
class Dictionary:
    def __init__(self):
        ### keeps unique terms (SORTED!)
        self.terms = self.loadTerms("terms.txt")
        self.idfs = [] ### IDF coefficients
        self.corM = [] ### a correlation matrix

    ### load terms
    def loadTerms(self, fileName):
        file = open(fileName,'r', encoding='utf-8-sig')
        k = [self.proc(s) for s in file.readlines()]
        k.sort()
        return k

    ### ignore it
    def proc(self, s):
        if s[-1] == '\n': return s[:-1]
        else: return s
    
    def computeIDFs(self, documents):
        self.idfs = []
        for term in self.terms:
            tf = 0
            for document in documents:
                if term in document.tokens:
                    tf += 1
            self.idfs.append(np.log(len(documents)/(1+tf)))
    
    def getUnionLength(self, v1, v2):
        return len(list(set(v1) | set(v2)))
        
    def getIntersectionLength(self, v1, v2):
        return len(list(set(v1) & set(v2)))
    
    def getJaccardIndex(self, v1, v2):
        return self.getIntersectionLength(v1,v2) / self.getUnionLength(v1,v2)
        
#     def computeCorM(self, documents):
#         self.corM = np.zeros((len(documents), len(documents)))
#         for i,document1 in enumerate(documents):
#             for j, document2 in enumerate(documents):
#                 if i == j:
#                     self.corM[i][j] = -1
#                 else:
#                     self.corM[i][j] = self.getJaccardIndex(document1.tokens, document2.tokens)

    def computeCorM(self, documents):
        self.corM = []
        tfM = []
        for document in documents:
            tfM.append(document.tf)
        self.corM = np.matmul(np.transpose(tfM), tfM)
        for i in range(len(self.corM)):
            self.corM[i][i] = -1
        print(self.corM)
        

### SOME DEBUG
dictionary = Dictionary()
print(dictionary.terms[:50])

['aaai', 'about', 'academic', 'access', 'acquired', 'acquisition', 'action', 'activity', 'actual', 'adaptive', 'add', 'advance', 'agricultural', 'aha', 'aim', 'alert', 'algorithm', 'all', 'analysis', 'and', 'announcement', 'answer', 'anyone', 'application', 'applied', 'apply', 'applying', 'approach', 'approache', 'april', 'archive', 'are', 'area', 'areas', 'article', 'artificial', 'asked', 'august', 'author', 'automated', 'automatically', 'autonomous', 'available', 'awards', 'backend', 'backgammon', 'baldi', 'based', 'basic', 'bayesian']


1.2) Load files: here we load some example collection of documents. RAW_DOCUMENTS = just strings. Check if the documents are loaded correctly (e.g., print RAW_DOCUMENTS[0])

In [3]:
RAW_DOCUMENTS = cm.loadDocuments("documents.txt")
### SOME DEBUG
print(RAW_DOCUMENTS[0])

David W. Aha:  Machine Learning Page
 Machine Learning Resources. Suggestions welcome. ... (WizRule); ZDM Scientific
 Ltd. Conference Announcements. Courses on Machine Learning. Data Repositories. ... 
 Description: Comprehensive machine learning resources from Applications to Tutorials.



In [4]:
### SOME DEBUG, JUST RUN; check if (a) common.py is imported correctly and (b) 
### tokens are correctly derived from some document (e.g., RAW_DOCUMENTS[0])
print(cm.simpleTextProcessing(RAW_DOCUMENTS[0], re))

['david', 'aha', 'machine', 'learning', 'page', 'machine', 'learning', 'resource', 'suggestion', 'welcome', 'wizrule', 'zdm', 'scientific', 'ltd', 'conference', 'announcement', 'course', 'machine', 'learning', 'data', 'repository', 'description', 'comprehensive', 'machine', 'learning', 'resource', 'from', 'application', 'tutorials']


1.3) Get acquainted with the below class. 

In [5]:
class Document:
    def __init__(self, doc_id, raw_document, dictionary):
        self.doc_id = doc_id ### DOC ID, simply 0,1,2,3....
        self.raw_document = raw_document ### raw data, i.e., string data
        self.dictionary = dictionary # reference to the dictionary
        
        ### DOCUMENT REPRESENTATIONS
        self.tokens = cm.simpleTextProcessing(raw_document, re) ### get terms
        self.bow = [] # Bag Of Words (BOW - number of term occurences)
        self.tf = [] # TF representation
        self.tf_idf = [] # TF-IDF representation

    def normalize(self, arr, t_min, t_max):
        norm_arr = []
        diff = t_max - t_min
        diff_arr = max(arr) - min(arr)    
        for i in arr:
            temp = (((i - min(arr))*diff)/diff_arr) + t_min
            norm_arr.append(temp)
        return norm_arr
    
    def computeBOW_Representation(self):
        usedwords = []
        self.bow = []
        for word in self.dictionary.terms:
            self.bow.append(self.tokens.count(word))
    
    def computeTF_Representation(self):
        self.tf = self.normalize(self.bow, 0, 1)
    
    ### TODO - complete this method; it should compute a TFxIDF representation 
    ### (important: it should not be run before dictionary.idfs are not computed!)
    def computeTF_IDF_Representation(self):
        self.tf_idf = [a * b for a,b in zip(self.tf, self.dictionary.idfs)]
    
    def computeRepresentations(self):
        self.computeBOW_Representation()
        self.computeTF_Representation()
        self.computeTF_IDF_Representation()
    
documents = [Document(i, RAW_DOCUMENTS[i], dictionary) for i in range(len(RAW_DOCUMENTS))]
documents[0].computeRepresentations()

1.4) Compute IDFs here

In [6]:
### TODO COMPUTE IDFS HERE (FINISH THE PROPER METHOD OF THE DICTIONARY CLASS - DO NOT FORGET TO RE-RUN THE CELL)
dictionary.computeIDFs(documents)

### SOME DEBUG
res = [[dictionary.terms[i], dictionary.idfs[i]] for i in range(len(dictionary.terms)-1)]
res.sort(key = lambda x: x[1])
# LEAST COMMON WORDS - HIGH IDF
print(res[-5:])
# MOST COMMON WORDS - LOW IDF
print(res[:5])

[['workbench', 3.784189633918261], ['working', 3.784189633918261], ['www', 3.784189633918261], ['york', 3.784189633918261], ['young', 3.784189633918261]]
[['learning', 0.01142869582362285], ['machine', 0.01142869582362285], ['the', 0.43428554664365626], ['and', 0.7161366987846441], ['description', 0.8664189018339821]]


1.5) Compute the document representations (for each document run computeRepresentations())

In [7]:
for d in documents: d.computeRepresentations()
### SOME DEBUG (you should see some 4s - which terms are these?)
print(np.shape(documents[0].bow), np.shape(dictionary.terms))

(612,) (612,)


1.6) Finish the below method. It should compute and return a cosine similarity (v1 and v2 are two vectors - tf-idf representations)

In [8]:
### TODO 
def getSimilarity(v1, v2):
    x = np.dot(v1,v2)
    y = np.linalg.norm(v1) * np.linalg.norm(v2)
    return x/y

# print(getSimilarity([0.5,1,0.5], [1,0,1]))

1.7) Run the below script for different queries. getTopResults seeks for documents being the most similar/relevant to the query. Do you find the results satisfactory?

In [9]:
query = "machine learning"
# query = "academic research"
# query = "international conference"
# query = "international conference washington"

In [12]:
def getTopResults(query, documents, dictionary, similarity, top = 5):
    qd = Document(-1, query, dictionary)
    qd.computeRepresentations()
    ranks = [[d, getSimilarity(d.tf_idf, qd.tf_idf)] for d in documents]
    ranks.sort(key=lambda x: -x[1])
    for i in range(top):
        print("RANK = " + str(i+1) + " WITH SIMILARITY = " + str(ranks[i][1]) + " | DOC ID = " + str(ranks[i][0].doc_id))
        print(ranks[i][0].raw_document)
        print("")

getTopResults(query, documents, dictionary, getSimilarity, top = 5)

RANK = 1 WITH SIMILARITY = 0.009509748581362498 | DOC ID = 63
AI / Machine Learning Resources
 AI / Machine Learning Resources. General Machine Learning. The Journal
 of Machine Learning. MLnet Machine Learning Archive at GMD. The ... 


RANK = 2 WITH SIMILARITY = 0.007318885120039777 | DOC ID = 77
Machine Learning
 Machine Learning. Machine Learning Home Page (Editor) Machine Learning Home
 Page (Publisher) Machine Learning Online by Kluwer Academic Publishers: ... 


RANK = 3 WITH SIMILARITY = 0.0069272054253559315 | DOC ID = 34
Machine Learning
 Machine Learning. Related Sites. Machine Learning Resources courtesy
 of David Aha A Machine Learning Tutorial a good overview of the ... 


RANK = 4 WITH SIMILARITY = 0.006024548531684508 | DOC ID = 81
Oxford University Machine Learning Group
 Machine Learning at the Computing Laboratory. ... Logic Programming and
 Learning and Intelligent Systems. Other Machine Learning Groups. ... 


RANK = 5 WITH SIMILARITY = 0.005705201989583534 | DOC I

# 2) Query expansion

## 2.1) Correlation matrix (for 3.5)

2.1.1) Finish dictionary.computeCorM method (see class Dictionary). It should generate a correlation matrix (correlation between terms).

IMPORTANT: although corM[ i ][ i ] (for each i) should be 1.0, set it to -1.0

In [11]:
### TODO - COMPLETE THE computeCorM METHOD (see one of the first cells)
dictionary.computeCorM(documents)
print(np.shape(dictionary.corM))

[[-1.      0.      0.     ...  0.      0.      0.    ]
 [ 0.     -1.      0.     ...  0.0625  0.      0.    ]
 [ 0.      0.     -1.     ...  0.      0.      0.    ]
 ...
 [ 0.      0.0625  0.     ... -1.      0.      0.    ]
 [ 0.      0.      0.     ...  0.     -1.      0.    ]
 [ 0.      0.      0.     ...  0.      0.     -1.    ]]
(612, 612)


2.1.2) Finish the below method. For each term in the query (you must parse the query, see getTopResults() method), find another term which is the most correlated with the input term.

In [14]:
# query = "machine"
# query = "algorithm"
# query = "learning"
query = "conference"
# query = "research"
# query = "concept"

def suggestKeywords(query, dictionary):
    qd = Document(-1, query, dictionary)
    for qterm in qd.tokens:
        qTermPosInDict = dictionary.terms.index(qterm)
        bestMatchIndex = np.max(dictionary.corM[qTermPosInDict])
        bestMatch = dictionary.terms[int(bestMatchIndex)]
        print(bestMatchIndex, bestMatch)
        
suggestKeywords(query, dictionary)

6.451388888888889 action


# 2.2) Rocchio algorithm (for 4.0)

$\overrightarrow{q_{m}} = \alpha \overrightarrow{q} + \left(\beta \cdot \dfrac{1}{|D_{r}|} \sum_{\overrightarrow{D_j} \in D_{r}} \overrightarrow{D_j} \right) - \left(\gamma \cdot \dfrac{1}{|D_{nr}|} \sum_{\overrightarrow{D_j} \in D_{nr}} \overrightarrow{D_j} \right)$

2.2.1) Firstly, run the below code. Observe the results. Assume that we do not like the first and the second result (Docs 63 and 77). However, assume that docs 29 and 36 are satisfactory. Now, modfify the method. It should alter the query vector, according to Rocchio algorithm. Check the result for the above considered scenario (relevant docs = 29 and 36; not relevant = 63 and 77). Check the results for different values of alpha, beta, and gamma coefficients. 

In [ ]:
def getTopResults_Rocchio(query, 
                          documents, 
                          dictionary, 
                          similarity, 
                          rel_docs = [29, 36],
                          nrel_docs = [63, 77],
                          alpha = 0.5,
                          beta = 0.3,
                          gamma = 0.2,
                          top = 10):
    qd = Document(-1, query, dictionary)
    qd.computeRepresentations()
    ##### TODO: MODIFY qd.tf_idf HERE
    
    #####
    ranks = [[d, getSimilarity(d.tf_idf, qd.tf_idf)] for d in documents]
    ranks.sort(key=lambda x: -x[1])
    for i in range(top):
        print("RANK = " + str(i+1) + " WITH SIMILARITY = " + str(ranks[i][1]) + " | DOC ID = " + str(ranks[i][0].doc_id))
        print(ranks[i][0].raw_document)
        print("")

getTopResults_Rocchio("machine learning", documents, dictionary, getSimilarity, top = 10)

# 2.3) WordNet (for 5.0)

2.3.1) Installation

http://www.nltk.org/install.html

import nltk 

nltk.download()

https://www.nltk.org/data.html

In [ ]:
from nltk.corpus import wordnet as wn

Definition: synset = (from wiki) (information science) A set of one or more synonyms that are interchangeable in some context without changing the truth value of the proposition in which they are embedded.

2.3.2) Display sysents for "machine"

In [ ]:
wn.synsets('machine')

2.3.3) Display all definitions (.definition()) for synsets (machine)

In [ ]:
#TODO

2.3.4) For each synset (machine), display its hypernym (a word with a broad meaning constituting a category into which words with more specific meanings fall; a superordinate. For example, colour is a hypernym of red).

In [ ]:
#TODO

See: http://www.nltk.org/howto/wordnet.html
for more examples